# Text To Speech Translator

## Imports And Libaries 

In [ ]:
    pip list # All The Pip installs have been made 

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as pltlist_physical_devices
import time
import mediapipe as mp

## Keypoints and values for evulation.

In [ ]:
mp_holistic = mp.solutions.holistic #Skeleton Model,
mp_drawing = mp.solutions.drawing_utils #Drawing Points On the Skeleton 

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Allow to convert Colour of image going from BGR to RGC
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction on image 
    image.flags.writeable = True                   # Then Write to Image 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #Going Back to BGR
    return image, results


In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Right Hand  

In [ ]:
#Creating Camera for Testing
cap = cv2.VideoCapture(0)

#Using Full resolution of caemra.
#Set FPS
fps_time = time.time()
#Set Mediapipe Model To start tracking and then Go off that Point 
with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened:
        #Reading video
        ret, frame = cap.read()
    
        #Making Detections:
        image, results = mediapipe_detection(frame,holistic)
        #Draw Sketelon 
        draw_landmarks(image,results)
        
        # Calculate FPS
        current_time = time.time()
        fps = 1 / (current_time - fps_time)
        fps_time = current_time
    
        # Display the FPS on the video feed
        cv2.putText(image, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        #Show on Laptop
        cv2.imshow('Feed' , image)
        #Quit When Q is pressed ( Hopefully Gracefully) 
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

## Getting Values From MediaPipe to Train & Create Folders for data


In [ ]:
rhsize = 21*3
print(rhsize)

In [ ]:
def extract_keypoints(results): # Extract The X Y Z Values of All the Points. If not on screen Replace with 0
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(rhsize)
    return np.concatenate([rh])

In [ ]:
if(rhsize == len(extract_keypoints(results))):
    print(len(extract_keypoints(results)))
else:
    print("Error")

In [10]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data')

# Actions that we try to detect
actions = np.array(['Hello' , 'I Love You' , 'Good' ,"Bad" , 'Yes', 'No'])
no_sequences = 30

#30 Frames ( 10 FPS Video - 2 Secs A Vid)
sequence_length = 20


In [11]:
#Make a Folder for each Word
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## Making Videos For Dataset 


# # Using Data Collected For For Machine LEearning to create Labels and Features 

In [13]:
cap = cv2.VideoCapture(0)


# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Wait for spacebar press to start the action collection
        cv2.putText(image, 'Press SPACE to start collecting frames for {}'.format(action), (15, 12), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
        cv2.imshow('OpenCV Feed', image)
        
        while True:
            key = cv2.waitKey(1) & 0xFF
            if key == ord(' '):  # Spacebar press
                break
                
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
pip install scikit-learn

In [14]:
#Preforcessing Imports 
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [15]:
# Define the actions you want to include in the training data
selected_actions = actions

# Create a label map for the selected actions
label_map = {label: num for num, label in enumerate(selected_actions)}

sequences = []
labels = []

for action in selected_actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

# Convert sequences and labels to NumPy arrays
X = np.array(sequences)
y = to_categorical(labels).astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6)

# Check the shape of the test labels
print("Shape of y_test:", y_test.shape)


Shape of y_test: (108, 6)


## Train Neutral network Then predict

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Assuming you've already filtered your dataset to include only the first 3 actions
# Define your log directory for TensorBoard
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

# Create a Sequential model
model = Sequential()

# Add LSTM layers
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(20, 63)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

# Add fully connected layers
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

# Output layer with activation 'softmax'; use the number of classes in your dataset
# In your case, it's the number of unique actions in the filtered dataset
num_classes = len(selected_actions)
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Assuming you have already prepared your X_train and y_train using the filtered dataset
# Train the model
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

# Display a summary of the model architecture
model.summary()


Epoch 1/2000
3/3 [==============================] - 3s 51ms/step - loss: 1.7931 - categorical_accuracy: 0.2639
Epoch 2/2000
3/3 [==============================] - 0s 42ms/step - loss: 1.7843 - categorical_accuracy: 0.1806
Epoch 3/2000
3/3 [==============================] - 0s 44ms/step - loss: 1.7752 - categorical_accuracy: 0.1944
Epoch 4/2000
3/3 [==============================] - 0s 48ms/step - loss: 1.7692 - categorical_accuracy: 0.1944
Epoch 5/2000
3/3 [==============================] - 0s 49ms/step - loss: 1.7457 - categorical_accuracy: 0.2917
Epoch 6/2000
3/3 [==============================] - 0s 53ms/step - loss: 1.7398 - categorical_accuracy: 0.3889
Epoch 7/2000
3/3 [==============================] - 0s 51ms/step - loss: 1.6995 - categorical_accuracy: 0.4444
Epoch 8/2000
3/3 [==============================] - 0s 51ms/step - loss: 1.5828 - categorical_accuracy: 0.3333
Epoch 9/2000
3/3 [==============================] - 0s 50ms/step - loss: 1.5957 - categorical_accuracy: 0.2778
E

 ## Make Prediction and Save Weights 

In [19]:
model.save('0.5Acc.h5')

/home/f1z/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()

yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)
print(accuracy_score(ytrue, yhat))


NameError: name 'model' is not defined

In [18]:
model.load_weights('0.5Acc.h5')

In [19]:
from sklearn.metrics import accuracy_score

# Assuming you have trained a neural network model 'model' and made predictions 'y_pred'
y_true = np.argmax(y_test, axis=1)  # If your y_test is one-hot encoded, convert it to class indices
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert model predictions to class indices

# Calculate the accuracy score
accuracy = accuracy_score(y_true, y_pred_classes)
print("Accuracy Score:", accuracy)


4/4 [==============================] - 0s 30ms/step
Accuracy Score: 0.8240740740740741


In [16]:
from tensorflow.keras.models import load_model

# Load the model from the 'My_model.h5' file
model = load_model('0.5Acc.h5')

# Now you can use the 'model' for predictions or further tasks

2023-11-05 01:53:53.396068: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [16]:
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        # Define the coordinates of the top-left and bottom-right corners
        top_left = (0, 60 + num * 40)
        bottom_right = (int(prob * 100), 90 + num * 40)
        # Draw the rectangle using the defined coordinates
        cv2.rectangle(output_frame, top_left, bottom_right, (255,255,255), -1)
        cv2.putText(output_frame, actions[num], (0, 85 + num * 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame

In [17]:
from gtts import gTTS
import playsound

def text_to_speech(text):
    # Convert the input text to speech
    tts = gTTS(text)

    # Save the speech as an MP3 file
    tts.save("output.mp3")

    # Play the audio file
    playsound.playsound("output.mp3")

    # Optionally, you can remove the temporary audio file after playing
    # os.remove("output.mp3")

In [25]:

# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                        
                else:
                    sentence.append(actions[np.argmax(res)])
                    

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            if res[np.argmax(res)] > threshold:
                action = actions[np.argmax(res)]
                text_to_speech(action)
            if len(sentence) > 0:
                if action != sentence[-1]:
                    sentence.append(action)
                    text_to_speech(action)
            else:
                sentence.append(action)
                text_to_speech(action)
    
            if len(sentence) > 5:
                sentence = sentence[-5]
    
            # Viz probabilities
            colors = (255,255,255)
            image = prob_viz(res, actions, image, colors)
    
            # Convert the recognized action to speech
            

            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows().append(actions[np.argmax(res)])
                        text_to_speech(action)
                else:
                    sentence.append(actions[np.argmax(res)])
                    text_to_speech(action)

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            if res[np.argmax(res)] > threshold:
                action = actions[np.argmax(res)]
            if len(sentence) > 0:
                if action != sentence[-1]:
                    sentence.append(action)
                    text_to_speech(action)
            else:
                sentence.append(action)
                text_to_speech(action)
    
            if len(sentence) > 5:
                sentence = sentence[-5]
    
            # Viz probabilities
            colors = (255,255,255)
            image = prob_viz(res, actions, image, colors)
    
            # Convert the recognized action to speech
            

            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [77]:
pip install gTTS playsound

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7035 sha256=49111c9775ec8967b9c72f621820e526f9b2df046eb66a93549c239a0f6002e7
  Stored in directory: /home/f1z/.cache/pip/wheels/90/89/ed/2d643f4226fc8c7c9156fc28abd8051e2d2c0de37ae51ac45c
Successfully built playsound
Note: you may need to restart the kernel to use updated packages.
